In [1]:
import os
os.chdir("/Users/juankostelec/Google_drive/Projects/taxGPT-database")

In [2]:
! pip install -e .

Obtaining file:///Users/juankostelec/Google_drive/Projects/taxGPT-database
  Preparing metadata (setup.py) ... done
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Attempting uninstall: taxgpt-database
    Found existing installation: taxgpt-database 0.1.0
    Uninstalling taxgpt-database-0.1.0:
      Successfully uninstalled taxgpt-database-0.1.0
  Running setup.py develop for taxgpt-database

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [13]:
import numpy as np
import pandas as pd
import os
import json
import sys
from urllib.parse import urlparse

# sys.path.append("/Users/juankostelec/Google_drive/Projects/taxGPT-database/app")
from app.utils import is_url_to_file, get_filetype
from app.parser.text_parser import FileProcessor

pd.set_option('display.max_colwidth', None)
DIR = "/Users/juankostelec/Google_drive/Projects/taxGPT-database/data"
reference_data_path = os.path.join(DIR,  "references.csv")
downloaded_data_path = os.path.join(DIR,  "downloaded_data_index.csv")
references = pd.read_csv(reference_data_path)
downloaded = pd.read_csv(downloaded_data_path)
print(len(downloaded))
downloaded["was_converted"] = downloaded["processed_filepath"].notna()
downloaded["was_chunked"] = downloaded["file_chunks_path"].notna()
downloaded["was_uploaded"] = downloaded["in_vector_db"].notna()

1501


## Referenced --> Downloaed

In [14]:
# Analyze first part of the pipeline
# Which rows of the reference data were not downloaded!!
print(references.columns)

# Check the rows of references data whose file_id is not present in the downloaded data
not_downloaded_rows = references[~references["file_id"].isin(downloaded["file_id"])]
not_downloaded_rows["is_reference_file"] = not_downloaded_rows["reference_href_clean"].apply(is_url_to_file)

# Keep only rows tha$t:
# a) details_href is not None
# b) details_href is None and reference_href_clean is not None and it's a file
not_downloaded_detailed_rows = not_downloaded_rows[not_downloaded_rows['details_href'].notna()]
not_downloaded_reference_rows = not_downloaded_rows[
    (not_downloaded_rows['details_href'].isna() & 
     not_downloaded_rows['reference_href_clean'].notna() & 
     (not_downloaded_rows['is_reference_file'] == True)
    )
]
print(not_downloaded_detailed_rows["is_scraped"].value_counts())
print(not_downloaded_reference_rows["is_scraped"].value_counts())
# Conclusions: We download all the relevant main references
# Conclusions: I have indeed tried to download all the files. If there is a mismatch, it is likely
# either because the file cannot be downloaded, or it is a repeat (i.e. several rows reference
# the same file

# Check the filetypes of the files that were not downloaded --> only .zip files
not_downloaded_detailed_rows["file_type"] = not_downloaded_detailed_rows["details_href"].apply(get_filetype)
data = not_downloaded_detailed_rows[not_downloaded_detailed_rows["is_scraped"] == False]
print(data["file_type"].value_counts())
print(data.query("file_type == 'unknown'")["details_href"].head(200))




Index(['area_name', 'area_desc', 'reference_name', 'reference_href',
       'reference_href_clean', 'details_section', 'details_section_text',
       'details_href_name', 'details_href', 'is_scraped', 'file_id',
       'used_download_href', 'actual_download_link',
       'actual_download_location', 'date_downloaded'],
      dtype='object')
True     549
False     60
Name: is_scraped, dtype: int64
Series([], Name: is_scraped, dtype: int64)
zip    60
Name: file_type, dtype: int64
Series([], Name: details_href, dtype: object)


/var/folders/y1/f5f_tzds45b44fhpyxx38qs40000gn/T/ipykernel_73469/3487620252.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_downloaded_rows["is_reference_file"] = not_downloaded_rows["reference_href_clean"].apply(is_url_to_file)
/var/folders/y1/f5f_tzds45b44fhpyxx38qs40000gn/T/ipykernel_73469/3487620252.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_downloaded_detailed_rows["file_type"] = not_downloaded_detailed_rows["details_href"].apply(get_filetype)


## Downloaded --> Converted

In [15]:
print(downloaded.columns)
not_converted_data = downloaded[~downloaded["was_converted"]]
print(not_converted_data["file_type"].value_counts())

# Let's explore HTML files that were not converted:
pdf_not_converted_files = not_converted_data[not_converted_data["file_type"] == "pdf"]
print(pdf_not_converted_files["downloaded_path"].head(10))

docs_not_converted_files = not_converted_data[not_converted_data["file_type"] == "docx"]
print(docs_not_converted_files["downloaded_path"].head(10))



Index(['file_id', 'filename', 'date_downloaded', 'area', 'subarea', 'section',
       'file_type', 'raw_filepath', 'processed_filepath', 'downloaded_path',
       'file_summary', 'file_chunks_path', 'in_vector_db', 'was_converted',
       'was_chunked', 'was_uploaded'],
      dtype='object')
pdf     16
docx     3
Name: file_type, dtype: int64
392                    /Users/juankostelec/Google_drive/Projects/taxGPT-database/data/raw_files/Vrednostni_limiti_v_posameznih_državah_članicah.pdf
473                                                       /Users/juankostelec/Google_drive/Projects/taxGPT-database/data/raw_files/Obrazec_VS-1.pdf
474                                                       /Users/juankostelec/Google_drive/Projects/taxGPT-database/data/raw_files/Obrazec_VP-1.pdf
475                                                       /Users/juankostelec/Google_drive/Projects/taxGPT-database/data/raw_files/Obrazec_HS-1.pdf
680                                                            

## Converted --> Chunked

In [16]:
converted_data = downloaded[downloaded["was_converted"]]
not_chunked_data = converted_data[~converted_data["was_chunked"]]
chunked_data = converted_data[converted_data["was_chunked"]]
print(not_chunked_data["file_type"].value_counts())

example = '/Users/juankostelec/Google_drive/Projects/taxGPT-database/data/chunked_files/Davčna_obravnava_odškodnin_"vse_v_enem"_po_5._točki_27._člena_ZDoh-2.txt'
print(os.path.splitext(os.path.basename(example))[0])



Series([], Name: file_type, dtype: int64)
Davčna_obravnava_odškodnin_"vse_v_enem"_po_5._točki_27._člena_ZDoh-2


## Chunked --> Uploaded

In [18]:
chunked_data = downloaded[downloaded["was_chunked"]]
not_uploaded_data = chunked_data[~chunked_data["was_uploaded"]]
print(not_uploaded_data["file_type"].value_counts())



html    405
docx    213
doc     161
pdf     101
xlsx     23
Name: file_type, dtype: int64


## Testing functions

In [ ]:
# Let's test how to better convert the html to md
import html2text
def convert_html_to_md(path, save_path):
    h = html2text.HTML2Text()
    h.ignore_links = True
    h.ignore_images = True
    with open(path, "r") as fin:
        markdown_content = h.handle(fin.read())
        with open(save_path, "w") as fout:
            fout.write(markdown_content)
    return save_path

# Use pandoc to convert html to markdown
# I want to ignore the image data
from markdownify import markdownify as md
def convert_html_to_md_markdownify(html_path, md_path):
    with open(html_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    markdown = md(html_content, strip=['img'])  # Stripping 'img' tags
    with open(md_path, 'w', encoding='utf-8') as file:
        file.write(markdown)

file_path = "/Users/juankostelec/Google_drive/Projects/taxGPT-database/data/raw_files/Dokument_02009R1072-20220221.html"
save_path = "/Users/juankostelec/Google_drive/Projects/taxGPT-database/data/converted_files/Dokument_02009R1072-20220221.md"

file_path = "/Users/juankostelec/Google_drive/Projects/taxGPT-database/data/raw_files/Pravilnik_o_vodenju_in_vzdrževanju_davčnega_registra.html"
save_path = "/Users/juankostelec/Google_drive/Projects/taxGPT-database/data/converted_files/Pravilnik_o_vodenju_in_vzdrževanju_davčnega_registra_test.md"
convert_html_to_md(file_path, save_path)

# save_path1 = "/Users/juankostelec/Google_drive/Projects/taxGPT-database/data/converted_files/Dokument_02009R1072-20220221_v2.md"
# convert_html_to_md_markdownify(file_path, save_path1)